In [4]:
import cv2
import numpy as np

# function to detect lanes
def lane_detection(image):
  try:
    # creating gray scale image
    gray_image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

    # creating canny image
    canny_image = cv2.Canny(gray_image, 50 , 150)

    # getting height and width of image
    h, w, c = image.shape

    # making a triangle to mask the canny image
    # you can change the points to wherever you want to have a different part of image
    triangle = np.array([[(int(w/5),h),(int(w*0.8)+20,h),(int(w/2+70),int(h/2)+100),(int(w/2-50),int(h/2)+100)]])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask,triangle,[255,255,255])
    mask = cv2.cvtColor(mask,cv2.COLOR_RGB2GRAY)
    masked_image = cv2.bitwise_and(canny_image, mask)

    #### making the lines out of canny masked image
    # explaining the parameters:
    # masked_image: our image
    # 5,np.pi/180: related to polar coordinates
    # 70: threshold value
    # np.array([]): optional placeholder for storing the output lines
    # minLineLength: lines shorter than this length will be rejected
    # maxLineGap: if the gap between two line segments is greater than this value, they will be treated as separate lines
    init_lines = cv2.HoughLinesP(masked_image, 5, np.pi/180, 70, np.array([]), minLineLength=1, maxLineGap=200)

    # Checking for valid lines
    lines = []
    for line in init_lines:
      x1, y1, x2, y2 = line.reshape(4)
      parameters = np.polyfit((x1,x2),(y1,y2),1)
      slope = parameters[0]
      intercept = parameters[1]
      if ((not 600 < x1 < 700) and not ((-0.5 < slope <0 and intercept>image.shape[1]/2) or (0 < slope <0.5 and intercept<image.shape[1]/2))) and not -0.5<slope<0.5:
        lines.append(line)

    # creating lines
    lines_image = np.zeros_like(image)
    if lines is not None:
      for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        # last two parameters are color and thickness of lines
        cv2.line(lines_image, (x1,y1), (x2,y2), (255,0,0), 10)
    image_with_some_lines = cv2.addWeighted(image, 0.8, lines_image, 1, 0)

    # checking which line is for which lane(right or left) by checking slope and intercept of lines
    left_fit = []
    right_fit = []
    for line in lines:
      x1, y1, x2, y2 = line.reshape(4)
      parameters = np.polyfit((x1,x2),(y1,y2),1)
      slope = parameters[0]
      intercept = parameters[1]
      if -1 < slope < 0:
        left_fit.append((slope,intercept))
      elif 0 < slope < 1:
        right_fit.append((slope,intercept))

    # detecting one line for each side
    left_fit_average = np.average(left_fit, axis=0)
    right_fit_average = np.average(right_fit, axis=0)
    slope,intercept = left_fit_average
    y1 = image.shape[0] - 20
    y2 = int(y1*3.5/5)
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    left_line = np.array([x1,y1,x2,y2])
    slope,intercept = right_fit_average
    y1 = image.shape[0] - 20
    y2 = int(y1*3.5/5)
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    right_line = np.array([x1,y1,x2,y2])
    final_lines = np.array([left_line, right_line])

    # drawing that two lines
    lines_image2 = np.zeros_like(image)
    for line in final_lines:
      x1, y1, x2, y2 = line.reshape(4)
      cv2.line(lines_image2, (x1,y1), (x2,y2), (255,0,0), 10)

    # output as an image
    final_image = cv2.addWeighted(image, 0.8, lines_image2, 1, 0)
    return final_image

  except:
    # if any error occured, this code gives a error print and gives back the initial image
    print('\n\nan error occurred!')
    return image

# using the function
image = cv2.imread('path to image.jpg')
final_image = lane_detection(image)
cv2.imwrite('final_image.jpg', final_image)

True